Testing functionalised script

In [12]:
import os
print(os.environ.get("BASE_DIR"))
BASE_DIR = os.environ.get("BASE_DIR", "/work/FAC/FGSE/IDYST/tbeucler/downscaling/")
import sys
import cartopy.crs as ccrs
import cartopy.feature as cfeature
sys.path.append("Scripts/Functions")
from pathlib import Path
import matplotlib.pyplot as plt
from pyproj import Transformer
import xarray as xr
import numpy as np

None


In [13]:
#Loading observations, predictions from two unets, reconstructed
#Observations for four variables 
data_obs_RhiresD= xr.open_dataset("../../data_1971_2023/HR_files_full/RhiresD_1971_2023.nc").sel(time=slice("2011-01-01", "2020-12-31"))
data_obs_TabsD= xr.open_dataset("../../data_1971_2023/HR_files_full/TabsD_1971_2023.nc").sel(time=slice("2011-01-01", "2020-12-31"))
data_obs_TminD= xr.open_dataset("../../data_1971_2023/HR_files_full/TminD_1971_2023.nc").sel(time=slice("2011-01-01", "2020-12-31"))
data_obs_TmaxD= xr.open_dataset("../../data_1971_2023/HR_files_full/TmaxD_1971_2023.nc").sel(time=slice("2011-01-01", "2020-12-31"))

In [14]:
#reconstructed datasets
data_path_reconstructed_precip = xr.open_dataset(str(BASE_DIR / "raw_data" / "Reconstruction_UniBern_1763_2020" / "precip_1763_2020.nc")).sel(time=slice("2011-01-01", "2020-12-31"))
data_path_reconstructed_temp = xr.open_dataset(str(BASE_DIR / "raw_data" / "Reconstruction_UniBern_1763_2020" / "temp_1763_2020.nc")).sel(time=slice("2011-01-01", "2020-12-31"))
data_path_reconstructed_tmin = xr.open_dataset(
    str(BASE_DIR / "raw_data" / "Reconstruction_UniBern_1763_2020" / "tmin_1763_2020.nc")
).sel(time=slice("2011-01-01", "2020-12-31"))
data_path_reconstructed_tmax = xr.open_dataset(
    str(BASE_DIR / "raw_data" / "Reconstruction_UniBern_1763_2020" / "tmax_1763_2020.nc")
).sel(time=slice("2011-01-01", "2020-12-31"))

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [24]:
#Predictions from 1971 time series 
data_path_Unet_1971_RhiresD= xr.open_dataset(str(BASE_DIR/"sasthana"/"Downscaling"/"Downscaling_Models"/"models_UNet"/"UNet_Deterministic_Training_Dataset"/"downscaled_predictions_2011_2020_ds.nc"))["RhiresD"]
data_path_Unet_1971_TabsD= xr.open_dataset(str(BASE_DIR/"sasthana"/"Downscaling"/"Downscaling_Models"/"models_UNet"/"UNet_Deterministic_Training_Dataset"/"downscaled_predictions_2011_2020_ds.nc"))["TabsD"]
data_path_Unet_1971_TminD= xr.open_dataset(str(BASE_DIR/"sasthana"/"Downscaling"/"Downscaling_Models"/"models_UNet"/"UNet_Deterministic_Training_Dataset"/"downscaled_predictions_2011_2020_ds.nc"))["TminD"]
data_path_Unet_1971_TmaxD= xr.open_dataset(str(BASE_DIR/"sasthana"/"Downscaling"/"Downscaling_Models"/"models_UNet"/"UNet_Deterministic_Training_Dataset"/"downscaled_predictions_2011_2020_ds.nc"))["TmaxD"]


In [ ]:
###data_path_Unet_1771= xr.open_dataset(str(BASE_DIR/"sasthana"/"Downscaling"/"Downscaling_Models"/"models_UNet"/"UNet_Deterministic_Pretraining_Dataset"/"downscaled_predictions_1771_2020_ds.nc"))

In [26]:
city_coords_Zürich = (47.3769, 8.5417)  # Zürich
city_coords_Geneva = (46.2044, 6.1432)  # Geneva
city_coords_Locarno = (47.5596, 7.5886)  # Locarno

For Zürich

for precipitation

In [30]:
def plot_city_pdf(city_coords, obs, unet, recon, city_name="City"):
    lat = obs['lat'].values
    lon = obs['lon'].values
    lon2d, lat2d = np.meshgrid(lon, lat)
    # euclidean distance
    dist = np.sqrt((lat2d - city_coords[0])**2 + (lon2d - city_coords[1])**2)
    lat_idx, lon_idx = np.unravel_index(np.argmin(dist), dist.shape)
    obs_series = obs[:, lat_idx, lon_idx].values.flatten()
    unet_series = unet[:, lat_idx, lon_idx].values.flatten()
    recon_series = recon[:, lat_idx, lon_idx].values.flatten()

    obs_series = obs_series[~np.isnan(obs_series)]
    unet_series = unet_series[~np.isnan(unet_series)]
    recon_series = recon_series[~np.isnan(recon_series)]

    plt.figure(figsize=(7,5))
    plt.hist(obs_series, bins=50, density=True, alpha=0.5, label="Obs (RhiresD)")
    plt.hist(unet_series, bins=50, density=True, alpha=0.5, label="UNet 1971 (RhiresD)")
    plt.hist(recon_series, bins=50, density=True, alpha=0.5, label="Recon (precip)")
    plt.title(f"PDF at {city_name} ({city_coords[0]:.2f}, {city_coords[1]:.2f})")
    plt.xlabel("Precipitation")
    plt.ylabel("Density")
    plt.legend()
    plt.tight_layout()
    plt.show()

Usage

In [31]:

city_coords = (47.3769, 8.5417)
plot_city_pdf(
    city_coords,
    data_obs_RhiresD["RhiresD"],
    data_path_Unet_1971_RhiresD,
    data_path_reconstructed_precip["precip"],
    city_name="Zürich"
)

: 